# Optical Flow and the Lucas-Kanade Method
This is a fundamental shift from finding features in a single image to tracking how they move across a sequence of frames.

---

## 1. Optical Flow: The Big Picture

Optical flow is the pattern of apparent motion of objects, surfaces, and edges in a visual scene caused by the relative motion between an observer and a scene.

### The Key Assumption: Brightness Constancy

For Optical Flow to work, we assume that the intensity (brightness) of a moving pixel does not change between Frame 1 and Frame 2.

$$I(x, y, t) = I(x+dx, y+dy, t+dt)$$

This leads to the **Optical Flow Constraint Equation**:
$$f_x u + f_y v + f_t = 0$$

* $f_x, f_y$: Image gradients (change in space).
* $f_t$: Temporal gradient (change in time).
* $u, v$: The velocity vector we want to find.

**The Problem:** We have one equation but two unknowns ( and ). This is known as the **Aperture Problem**.

---

## 2. The Lucas-Kanade Method

To solve the Aperture Problem, Lucas and Kanade (1981) proposed that we assume all pixels in a small local window (e.g., 3X3) move with the **same velocity**.

Instead of one equation, we now have 9 equations for a 3X3 patch. We solve this using a "Least Squares" approach, which looks very familiar to the **Harris Corner Detector** you just learned:

$$\begin{bmatrix} \sum I_x^2 & \sum I_x I_y \\ \sum I_x I_y & \sum I_y^2 \end{bmatrix} \begin{bmatrix} u \\ v \end{bmatrix} = - \begin{bmatrix} \sum I_x I_t \\ \sum I_y I_t \end{bmatrix}$$

> **Insight:** If the matrix on the left is invertible (meaning it has two large eigenvalues), the movement  can be solved reliably. This is exactly why we use **Shi-Tomasi "Good Features to Track"** to initialize Lucas-Kanade!

---

## 3. The KLT Algorithm (Kanade-Lucas-Tomasi)

The KLT algorithm is the standard implementation. It combines the previous concepts into a robust pipeline:

1. **Detection:** Use Shi-Tomasi to find "trackable" corners (the best features).
2. **Tracking:** Use the Lucas-Kanade method to find where those corners moved in the next frame.
3. **Refinement:** Repeat for every frame, discarding features that become too distorted.

---

## 4. Implementation in Python

OpenCV provides `cv2.calcOpticalFlowPyrLK`, which includes an **image pyramid** to handle large motions (just like SIFT and SURF use pyramids to handle scale).
---

### Summary of Tracking Methods

| Method | Core Idea | Best For... |
| --- | --- | --- |
| **Lucas-Kanade** | Local differential method (gradients) | Small, consistent motion of points. |
| **Mean Shift** | Finding the "peak" of a color histogram | Large objects with a distinct color. |
| **Kalman Filter** | Predicting future position based on velocity/physics | Objects that might be briefly occluded. |

In [ ]:
import numpy as np
import cv2

# 1. Setup Video Capture (0 for webcam)
cap = cv2.VideoCapture(0)

# 2. Parameters for Shi-Tomasi Corner Detection
feature_params = dict(maxCorners=100,
                       qualityLevel=0.3,
                       minDistance=7,
                       blockSize=7)

# 3. Parameters for Lucas-Kanade Optical Flow
# winSize: size of the search window at each pyramid level
# maxLevel: 0-based maximal pyramid level number
lk_params = dict(winSize=(15, 15),
                  maxLevel=2,
                  criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# 4. Initialize: Take first frame and find corners
ret, old_frame = cap.read()
if not ret:
    print("Failed to read video")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes (tracking lines)
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 5. Calculate Optical Flow (Lucas-Kanade)
    # This finds where p0 (old points) moved in frame_gray (new frame)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # 6. Select "good" points (status 1 means successfully tracked)
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

    # 7. Visualization: Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        # Draw line between old and new position
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        # Draw dot at new position
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2.imshow('KLT Tracker', img)

    # 8. Update for the next iteration
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    # Press 'q' to exit or 'r' to reset points
    key = cv2.waitKey(30) & 0xff
    if key == ord('q'):
        break
    elif key == ord('r'): # Reset tracking points
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        mask = np.zeros_like(old_frame)

cap.release()
cv2.destroyAllWindows()